In [8]:
import pandas as pd

df = pd.read_csv("hf://datasets/roman-bushuiev/MassSpecGym/data/MassSpecGym.tsv", sep="\t")
df = df[df["instrument_type"] == "Orbitrap"].drop_duplicates(["inchikey"])
df

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 86bb5962-3edc-45db-b45e-71da2358c037)')' thrown while requesting GET https://huggingface.co/datasets/roman-bushuiev/MassSpecGym/resolve/main/data/MassSpecGym.tsv
Retrying in 1s [Retry 1/5].


KeyboardInterrupt: 

In [ ]:
unique_counts = df.nunique()

summary = pd.DataFrame({
    "unique_values": unique_counts,
    "non_null_count": df.count(),
    "dtype": df.dtypes
})

print(summary)

                      unique_values  non_null_count    dtype
identifier                    22646           22646   object
mzs                           22604           22646   object
intensities                   22463           22646   object
smiles                        22642           22646   object
inchikey                      22646           22646   object
formula                       15225           22646   object
precursor_formula             15795           22646   object
parent_mass                   18152           22646  float64
precursor_mz                  18084           22646  float64
adduct                            2           22646   object
instrument_type                   1           22646   object
collision_energy               1895           15291  float64
fold                              3           22646   object
simulation_challenge              2           22646     bool


In [ ]:
df["fold"].value_counts()

fold
train    17220
val       2775
test      2651
Name: count, dtype: int64

In [ ]:
import numpy as np

df["mzs"] = df["mzs"].apply(lambda x: np.array(list(map(float, x.split(",")))))
df["intensities"] = df["intensities"].apply(lambda x: np.array(list(map(float, x.split(",")))))

### Separate by folds

In [ ]:
df_train = df[df["fold"] == "train"]
df_val = df[df["fold"] == "val"]
df_test = df[df["fold"] == "test"]

df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

# Save the preprocessed data
df_train.to_csv("train_data.csv", index=False)
df_val.to_csv("val_data.csv", index=False)
df_test.to_csv("test_data.csv", index=False)

### Compute cosine similarity scores using MatchMS 

In [9]:
import pandas as pd
import numpy as np

df_train = pd.read_csv("train_data.csv")
df_val = pd.read_csv("val_data.csv")
df_test = pd.read_csv("test_data.csv")

for df in [df_train, df_val, df_test]:
    df["mzs"] = df["mzs"].apply(lambda x: np.array(list(map(float, x.strip("[]").split()))))
    df["intensities"] = df["intensities"].apply(lambda x: np.array(list(map(float, x.strip("[]").split()))))

In [10]:
import numpy as np
from matchms import Spectrum
from matchms.similarity import ModifiedCosine
from rdkit import Chem
from rdkit.Chem import rdFMCS

def similarity_function(spectrum_1, spectrum_2):
    """
    Calculate the modified cosine similarity between two spectra.
    """
    # Use factory to construct a similarity function
    modified_cosine = ModifiedCosine(tolerance=0.01)
    score = modified_cosine.pair(spectrum_1, spectrum_2)
    return float(score['score'])

def compute_similarity(row, spectrum_ref):
    spectrum_candidate = Spectrum(mz=row["mzs"],
                          intensities=row["intensities"],
                          metadata={"precursor_mz": row["precursor_mz"]})
    score = similarity_function(spectrum_ref, spectrum_candidate)
    return score

def compute_mcs_percent(row, mol_ref):
    """
    MCS % based on number of atoms in the reference molecule.
    """
    mol_candidate = Chem.MolFromSmiles(row["smiles"])
    mcs = rdFMCS.FindMCS([mol_candidate, mol_ref], completeRingsOnly=True)
    if mcs.smartsString == "":
        return 0.0

    common = Chem.MolFromSmarts(mcs.smartsString)
    common_atoms = common.GetNumAtoms()
    total_atoms_ref = mol_ref.GetNumAtoms()

    return (common_atoms / total_atoms_ref) * 100

def select_most_similar_spectra(row, table, threshold=0.8):
    """
    Select the most similar spectra from the dataframe.
    """
    df = table.copy()
    spectrum = Spectrum(mz=row["mzs"],
                        intensities=row["intensities"],
                        metadata={"precursor_mz": row["precursor_mz"]})
    mol = Chem.MolFromSmiles(row["smiles"])
    df['matchms_similarity'] = df.apply(lambda r: compute_similarity(r, spectrum), axis=1)
    df['mcs_percent'] = df.apply(lambda r: compute_mcs_percent(r, mol), axis=1)
    df.loc[df['inchikey'] == row["inchikey"], 'matchms_similarity'] = -1.0
    df.loc[df['inchikey'] == row["inchikey"], 'mcs_percent'] = -1.0
    df["combined_score"] = df[["matchms_similarity", "mcs_percent"]].min(axis=1)
    df.sort_values(by="combined_score", ascending=False, inplace=True)
    return pd.Series([df.iloc[0]["inchikey"], 
                      df.iloc[0]["matchms_similarity"], 
                      df.iloc[0]["mcs_percent"],
                      df.iloc[0]["combined_score"]
                    ])

def similarity_search(df):
    df[["inchikey_similar",
        "matchms_similarity",
        "mcs_percent",
        "combined_score"]] = df.apply(lambda row: select_most_similar_spectra(row, df), axis=1)
    return df

In [15]:
import numpy as np
import pandas as pd
from matchms import Spectrum
from matchms.similarity import ModifiedCosine
from rdkit import Chem
from rdkit.Chem import rdFMCS
from tqdm import tqdm

# Function to compute matchms similarity between two spectra
def similarity_function(spectrum_1, spectrum_2):
    modified_cosine = ModifiedCosine(tolerance=0.01)
    score = modified_cosine.pair(spectrum_1, spectrum_2)
    return float(score['score'])

# Function to compute MCS % between two molecules
def compute_mcs_percent(mol1, mol2):
    mcs = rdFMCS.FindMCS([mol1, mol2], completeRingsOnly=True)
    if mcs.smartsString == "":
        return 0.0
    common = Chem.MolFromSmarts(mcs.smartsString)
    return (common.GetNumAtoms() / mol1.GetNumAtoms()) * 100

# Precompute all spectra into Spectrum objects
def prepare_spectra(df):
    spectra = []
    for idx, row in df.iterrows():
        spectra.append(
            Spectrum(mz=row["mzs"],
                     intensities=row["intensities"],
                     metadata={"precursor_mz": row["precursor_mz"]})
        )
    return spectra

# Precompute all molecules into RDKit Mol objects
def prepare_mols(df):
    mols = []
    for idx, row in df.iterrows():
        mols.append(Chem.MolFromSmiles(row["smiles"]))
    return mols

# Main function
def fast_similarity_search(df, top_k=5):
    # Step 1: Prepare spectra and mols
    print("Preparing spectra and molecules...")
    spectra = prepare_spectra(df)
    mols = prepare_mols(df)
    
    # Step 2: Precompute matchms similarities
    print("Computing spectral similarities...")
    matchms_matrix = np.zeros((len(df), len(df)))
    for i in tqdm(range(len(spectra))):
        for j in range(i+1, len(spectra)):
            score = similarity_function(spectra[i], spectra[j])
            matchms_matrix[i, j] = score
            matchms_matrix[j, i] = score  # symmetric

    results = []

    # Step 3: For each spectrum, find top K candidates
    print("Selecting best candidates...")
    for idx in tqdm(range(len(df))):
        similarities = matchms_matrix[idx]
        # Ignore self-comparison
        similarities[idx] = -1.0
        # Find top K indices
        top_indices = np.argsort(similarities)[-top_k:]
        
        best_score = -1
        best_candidate_idx = None
        best_matchms = None
        best_mcs = None

        for candidate_idx in top_indices:
            sim = similarities[candidate_idx]
            if sim < 0.3:  # Optional: minimal similarity threshold
                continue

            # Compute MCS %
            mcs_percent = compute_mcs_percent(mols[idx], mols[candidate_idx])

            combined_score = min(sim, mcs_percent / 100)

            if combined_score > best_score:
                best_score = combined_score
                best_candidate_idx = candidate_idx
                best_matchms = sim
                best_mcs = mcs_percent

        if best_candidate_idx is not None:
            results.append({
                "query_inchikey": df.iloc[idx]["inchikey"],
                "similar_inchikey": df.iloc[best_candidate_idx]["inchikey"],
                "matchms_similarity": best_matchms,
                "mcs_percent": best_mcs,
                "combined_score": best_score
            })
        else:
            results.append({
                "query_inchikey": df.iloc[idx]["inchikey"],
                "similar_inchikey": None,
                "matchms_similarity": None,
                "mcs_percent": None,
                "combined_score": None
            })

    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
import numpy as np
import pandas as pd
from matchms import Spectrum
from matchms.similarity import ModifiedCosine
from rdkit import Chem
from rdkit.Chem import rdFMCS
from tqdm import tqdm

# Function to compute matchms similarity between two spectra
def similarity_function(spectrum_1, spectrum_2):
    modified_cosine = ModifiedCosine(tolerance=0.01)
    score = modified_cosine.pair(spectrum_1, spectrum_2)
    return float(score['score'])

# Function to compute MCS % between two molecules
def compute_mcs_percent(mol1, mol2):
    mcs = rdFMCS.FindMCS([mol1, mol2], completeRingsOnly=True)
    if mcs.smartsString == "":
        return 0.0
    common = Chem.MolFromSmarts(mcs.smartsString)
    return (common.GetNumAtoms() / mol1.GetNumAtoms()) * 100

# Precompute all spectra into Spectrum objects
def prepare_spectra(df):
    spectra = []
    for idx, row in df.iterrows():
        spectra.append(
            Spectrum(mz=row["mzs"],
                     intensities=row["intensities"],
                     metadata={"precursor_mz": row["precursor_mz"]})
        )
    return spectra

# Precompute all molecules into RDKit Mol objects
def prepare_mols(df):
    mols = []
    for idx, row in df.iterrows():
        mols.append(Chem.MolFromSmiles(row["smiles"]))
    return mols

# Main function
def fast_similarity_search(df, top_k=5):
    # Step 1: Prepare spectra and mols
    print("Preparing spectra and molecules...")
    spectra = prepare_spectra(df)
    mols = prepare_mols(df)
    
    # Step 2: Precompute matchms similarities
    print("Computing spectral similarities...")
    matchms_matrix = np.zeros((len(df), len(df)))
    for i in tqdm(range(len(spectra))):
        for j in range(i+1, len(spectra)):
            score = similarity_function(spectra[i], spectra[j])
            matchms_matrix[i, j] = score
            matchms_matrix[j, i] = score  # symmetric

    results = []

    # Step 3: For each spectrum, find top K candidates
    print("Selecting best candidates...")
    for idx in tqdm(range(len(df))):
        similarities = matchms_matrix[idx]
        # Ignore self-comparison
        similarities[idx] = -1.0
        # Find top K indices
        top_indices = np.argsort(similarities)[-top_k:]
        
        best_score = -1
        best_candidate_idx = None
        best_matchms = None
        best_mcs = None

        for candidate_idx in top_indices:
            sim = similarities[candidate_idx]
            if sim < 0.3:  # Optional: minimal similarity threshold
                continue

            # Compute MCS %
            mcs_percent = compute_mcs_percent(mols[idx], mols[candidate_idx])

            combined_score = min(sim, mcs_percent / 100)

            if combined_score > best_score:
                best_score = combined_score
                best_candidate_idx = candidate_idx
                best_matchms = sim
                best_mcs = mcs_percent

        if best_candidate_idx is not None:
            results.append({
                "query_inchikey": df.iloc[idx]["inchikey"],
                "similar_inchikey": df.iloc[best_candidate_idx]["inchikey"],
                "matchms_similarity": best_matchms,
                "mcs_percent": best_mcs,
                "combined_score": best_score
            })
        else:
            results.append({
                "query_inchikey": df.iloc[idx]["inchikey"],
                "similar_inchikey": None,
                "matchms_similarity": None,
                "mcs_percent": None,
                "combined_score": None
            })

    results_df = pd.DataFrame(results)
    return results_df

In [18]:
import numpy as np
import pandas as pd
from matchms import Spectrum
from matchms.similarity import ModifiedCosine
from rdkit import Chem
from rdkit.Chem import rdFMCS
from tqdm import tqdm
from joblib import Parallel, delayed

def similarity_function(spectrum_1, spectrum_2):
    modified_cosine = ModifiedCosine(tolerance=0.01)
    score = modified_cosine.pair(spectrum_1, spectrum_2)
    return float(score['score'])

def compute_mcs_percent(mol1, mol2):
    mcs = rdFMCS.FindMCS([mol1, mol2], completeRingsOnly=True)
    if mcs.smartsString == "":
        return 0.0
    common = Chem.MolFromSmarts(mcs.smartsString)
    return (common.GetNumAtoms() / mol1.GetNumAtoms()) * 100

def process_single_query(idx, df, spectra, mols, top_k=5):
    ref_spectrum = spectra[idx]
    ref_mol = mols[idx]
    ref_inchikey = df.iloc[idx]["inchikey"]

    similarities = []
    for j in range(len(df)):
        if j == idx:
            continue
        sim = similarity_function(ref_spectrum, spectra[j])
        similarities.append((j, sim))

    # Sort and pick top_k
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]

    best_score = -1
    best_candidate_idx = None
    best_matchms = None
    best_mcs = None

    for candidate_idx, sim in similarities:
        if sim < 0.3:
            continue
        mcs_percent = compute_mcs_percent(ref_mol, mols[candidate_idx])
        combined_score = min(sim, mcs_percent / 100)

        if combined_score > best_score:
            best_score = combined_score
            best_candidate_idx = candidate_idx
            best_matchms = sim
            best_mcs = mcs_percent

    if best_candidate_idx is not None:
        return {
            "query_inchikey": ref_inchikey,
            "similar_inchikey": df.iloc[best_candidate_idx]["inchikey"],
            "matchms_similarity": best_matchms,
            "mcs_percent": best_mcs,
            "combined_score": best_score
        }
    else:
        return {
            "query_inchikey": ref_inchikey,
            "similar_inchikey": None,
            "matchms_similarity": None,
            "mcs_percent": None,
            "combined_score": None
        }

def full_similarity_search_parallel(df, top_k=5, n_jobs=8):
    print("Preparing spectra and mols...")
    spectra = [
        Spectrum(mz=row["mzs"], intensities=row["intensities"], metadata={"precursor_mz": row["precursor_mz"]})
        for _, row in df.iterrows()
    ]
    mols = [Chem.MolFromSmiles(row["smiles"]) for _, row in df.iterrows()]

    print("Running full parallel search...")
    results = Parallel(n_jobs=n_jobs)(
        delayed(process_single_query)(i, df, spectra, mols, top_k)
        for i in tqdm(range(len(df)))
    )
    return pd.DataFrame(results)

In [19]:
results = full_similarity_search_parallel(df_train, top_k=5, n_jobs=8)
results.to_csv("similarity_results_train.csv", index=False)

Preparing spectra and mols...
Running full parallel search...


  1%|          | 96/17220 [01:30<6:02:23,  1.27s/it]

KeyboardInterrupt: 

### Convert to .mgf

In [29]:
from matchms import Spectrum
from matchms.exporting import save_as_mgf
import json
import numpy as np

def to_1d_array(x):
    """Safely convert any array-like input to a clean 1D numpy array."""
    if isinstance(x, str):
        try:
            x = json.loads(x)
        except Exception:
            x = [float(x)]
    x = np.array(x).flatten()  # ← flatten ensures 1D structure
    return x

# Convert spectra to MGF format
def convert_to_mgf(row, fold="train"):
    """
    Convert a single row of spectral data into MGF format and save it as a file.

    Parameters:
    row (pd.Series): A row from the DataFrame containing spectral data. It must include
                     the columns 'mzs', 'intensities', 'identifier', 'inchikey', 'smiles',
                     and 'precursor_mz'.
    fold (str): The folder name where the MGF file will be saved. Default is "train".

    Returns:
    None
    """
    mzs = to_1d_array(row["mzs"])
    intensities = to_1d_array(row["intensities"])
    identifier = row["identifier"]
    inchikey = row["inchikey"]
    smiles = row["smiles"]
    precursor_mz = row["precursor_mz"]

    # Create a spectrum
    spectrum = Spectrum(
        mz=mzs,
        intensities=intensities,
        metadata={
            "id": inchikey,
            "identifier": identifier,
            "inchikey": inchikey,
            "smiles": smiles,
            "precursor_mz": precursor_mz,
            "charge": 1,
            "retention_time": 0.0,
            "rt": 0.0
        },
    )
    # Save as .mgf
    save_as_mgf(spectrum, f"{fold}/{inchikey}.mgf")

In [30]:
df_train.apply(lambda row: convert_to_mgf(row, fold="train"), axis=1)
df_val.apply(lambda row: convert_to_mgf(row, fold="val"), axis=1)
df_test.apply(lambda row: convert_to_mgf(row, fold="test"), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
2993    None
2994    None
2995    None
2996    None
2997    None
Length: 2998, dtype: object

In [31]:
import os

folders = ["train", "test", "val"]
for folder in folders:
    for filename in os.listdir(folder):
        if filename.endswith(".mgf"):
            filepath = os.path.join(folder, filename)
            with open(filepath, "r") as f:
                lines = f.readlines()
            
            # Заменяем строки
            new_lines = []
            for line in lines:
                if line.strip().startswith("RETENTION_TIME="):
                    value = line.strip().split("=", 1)[1]
                    new_lines.append(f"rt={value}\n")  # 👈 меняем на "rt="
                else:
                    new_lines.append(line)

            # Перезаписываем файл
            with open(filepath, "w") as f:
                f.writelines(new_lines)